In [ ]:
!pip install datasets
!pip install transformers
!pip install wandb
!pip install llama-recipes

In [ ]:
import datasets
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    use_cache=None,
    attn_implementation=None,
)

In [ ]:
model.device

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
user_prompt = '''Task:
You are a helpful legal assistant. Choose the correct option by performing legal reasoning while strictly adhering to the legal context below.

Legal Context:
The Due Process Clause of the Fourteenth Amendment prohibits a state from depriving a person of life, liberty, or property without due process of law. When a court enters a civil judgment against a person, it begins the process of taking the person’s property (usually in the form of money). Consequently, the due process clause requires the court to use a fair procedure in entering judgment. Certainly, one component of a fair procedure is to tell the defendant that the court is going to adjudicate her rights. Hence, the Due Process Clause requires a court to use a constitutionally adequate means of notifying the defendant that a lawsuit has been commenced against her. Typically, it is the plaintiff who does the legwork of serving process on the defendant. The statutes or court rules in every state contain detailed provisions governing how this notice of a lawsuit is provided to the defendant. Court rules may authorize various means of serving process. The most obvious is to deliver the initiating papers in the case to the defendant in person, called ‘‘personal service of process.’’ Other methods are often authorized as well, though they are less certain to actually inform the defendant about the suit. The service rules may provide, for example, that the papers may be left with someone at the defendant’s home or place of business or slipped under the door. Some authorize service to the defendant by certified or ordinary mail. In some circumstances, court rules or statutes may authorize service by publication, by printing a notice of the suit in the legal notices in the local newspaper for several weeks. Such publication notice is sometimes referred to as ‘‘constructive service,’’ as opposed to actual delivery of the papers to the defendant. Which of these methods is constitutionally adequate under the Due Process Clause? Justice Jackson’s wise opinion in Mullane v. Central Hanover Bank & Trust Co., 339 U.S. 306 (1950) does not provide a mechanical answer to that question, but instead establishes broad standards for constitutionally adequate notice. Mullane involved a device called a common trust fund, which allowed a bank to pool a group of small trusts into one common fund, which could be managed more efficiently. Income (or losses) from the trust would then be shared by each smaller trust in proportion to their contributions to the fund. Periodically, the trustee of the fund would file accounts with the court. If the court approved the accounts, its judgment would bar investors from suing the bank for mismanagement. Since this judicial accounting could deprive trust beneficiaries of property—the right to sue the bank for mismanagement—due process required adequate notice to them of the proceeding. The problem in Mullane was that it was difficult to ascertain who all the beneficiaries were. Beneficiaries currently entitled to receive income from the trust were known, and the bank had their addresses in its files. But most trusts have contingent beneficiaries, such as the children or heirs of a beneficiary if the primary beneficiary dies. The bank might not know who these beneficiaries were, and they could change periodically. Perhaps, with a large investment of time by lawyers and investigators, the bank could identify most them, but doing so would eat up the profits of the common trust fund. Many were contingent beneficiaries; that is, they had no current right to receive income from the fund but might become vested beneficiaries later under the terms of the individual trusts. So who did the bank have to tell about the suit and how? Mullane held that ‘‘an elementary and fundamental requirement of due process in any proceeding which is to be accorded finality is notice reasonably calculated, under all the circumstances, to apprise interested parties of the pendency of the action and afford them an opportunity to present their objections.’’ 339 U.S. at 314. What is reasonable in one circumstance would not necessarily satisfy due process in another. On the facts of Mullane, several factors reduced the risk of error if notice failed to reach all beneficiaries. First, the statute provided for the appointment of a guardian in the settlement proceeding to represent the interests of all beneficiaries. Second, even if all beneficiaries did not receive notice of the proceeding, many would, and would likely share and represent the interests of all beneficiaries in the settlement of the accounts. Consequently, the Court in Mullane did not require that each individual with an interest in the fund be given personal notice of the action. The question below probes what it did require.

(A notice of pendency of action, also known as a lis pendens, is a legal document that notifies the public of a judicial proceeding that affects real property ownership. It's usually filed in the county recorder's office where the property is located.)

Question:
In Mullane v. Central Hanover Bank & Trust Co., the Supreme Court held that 
				
Options:
A. every person whose interests may be affected by a judicial proceeding is entitled to at least mail notice of the proceeding. 
B. every person whose name and address could be ascertained through reasonable investigation must be given individual notice of the proceeding.
C. any person whose interests might be affected by the proceeding must be given notice by in-hand service of process. 
D. due to the large number of persons whose interests might be affected by the proceedings, notice by publication was sufficient on the facts of the case. 
E. None of the above is true.



While answering make sure to use the following format:
[explanation of your legal reasoning step by step as numbered points]'''

In [ ]:
batch = tokenizer(user_prompt, return_tensors="pt")

In [ ]:
batch = {k: v.to("cuda") for k, v in batch.items()}
with torch.no_grad():
    outputs = model.generate(
        **batch,
        max_new_tokens=1000, #The maximum numbers of tokens to generate
        do_sample=False, #Whether or not to use sampling ; use greedy decoding otherwise.
        top_p=1.0, # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
        temperature=0, # [optional] The value used to modulate the next token probabilities.
        min_length=None, #The minimum length of the sequence to be generated, input prompt + min_new_tokens
        use_cache=True, #[optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
        top_k=50, # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
        repetition_penalty=1.2, #The parameter for repetition penalty. 1.0 means no penalty.
        length_penalty=1, #[optional] Exponential penalty to the length that is used with beam-based generation.
        output_hidden_states= True, return_dict_in_generate=True,
    )
batch = {k: v.to("cpu") for k, v in batch.items()}

In [ ]:
output_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

In [ ]:
print(output_text[len(user_prompt):])